In [4]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup

Top 5 Prem Winners All-Time

In [5]:
# Based on the structure of the dataframe, we will assign points to home and away teams based on the FTR column

df = pd.read_csv("/Users/ojonyeagwu/Desktop/epl_data/data/premier-league-matches.csv")

# Points for match outcomes
points = {
    'H': (3, 0),  # (Home team points, Away team points)
    'D': (1, 1),
    'A': (0, 3)
}

# Apply points to each game
df['HomePoints'] = df['FTR'].map(lambda x: points[x][0])
df['AwayPoints'] = df['FTR'].map(lambda x: points[x][1])

# Aggregate points by team and season
home_points = df.groupby(['Season_End_Year', 'Home'])['HomePoints'].sum().reset_index()
away_points = df.groupby(['Season_End_Year', 'Away'])['AwayPoints'].sum().reset_index()

# Rename columns for merging
home_points.rename(columns={'Home': 'Team', 'HomePoints': 'Points'}, inplace=True)
away_points.rename(columns={'Away': 'Team', 'AwayPoints': 'Points'}, inplace=True)

# Combine home and away points
total_points = pd.concat([home_points, away_points])

# Sum points by team and season
total_points = total_points.groupby(['Season_End_Year', 'Team'])['Points'].sum().reset_index()

# Determine the champion for each season by finding the team with the most points
champions = total_points.loc[total_points.groupby('Season_End_Year')['Points'].idxmax()]

# Count the titles for each team
titles_count = champions['Team'].value_counts().reset_index()
titles_count.columns = ['Team', 'Titles']

titles_count
df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,HomePoints,AwayPoints
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H,3,0
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H,3,0
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H,3,0
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D,1,1
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A,0,3
...,...,...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H,3,0
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H,3,0
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H,3,0
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A,0,3


In [30]:
# Aggregate Home and Away Goals
home_goals = df.groupby('Home')['HomeGoals'].sum().reset_index()
away_goals = df.groupby('Away')['AwayGoals'].sum().reset_index()

# Rename columns for merging
home_goals.rename(columns={'Home': 'Team', 'HomeGoals': 'HomeGoalsScored'}, inplace=True)
away_goals.rename(columns={'Away': 'Team', 'AwayGoals': 'AwayGoalsScored'}, inplace=True)

# Combine home and away goals
total_goals = pd.merge(home_goals, away_goals, on='Team')
total_goals['TotalGoals'] = total_goals['HomeGoalsScored'] + total_goals['AwayGoalsScored']
total_goals= total_goals.sort_values(by=['TotalGoals'], ascending=False)

#New Stats
# Aggregate Home Goals Conceded and Away Goals Conceded
home_goals_conceded = df.groupby('Home')['AwayGoals'].sum().reset_index()
away_goals_conceded = df.groupby('Away')['HomeGoals'].sum().reset_index()

# Rename columns for clarity
home_goals_conceded.rename(columns={'Home': 'Team', 'AwayGoals': 'HomeGoalsConceded'}, inplace=True)
away_goals_conceded.rename(columns={'Away': 'Team', 'HomeGoals': 'AwayGoalsConceded'}, inplace=True)

# Combine home and away goals conceded
total_goals_conceded = pd.merge(home_goals_conceded, away_goals_conceded, on='Team')
total_goals_conceded['TotalGoalsConceded'] = total_goals_conceded['HomeGoalsConceded'] + total_goals_conceded['AwayGoalsConceded']

# Combine the goals scored and conceded dataframes
total_stats = pd.merge(total_goals, total_goals_conceded[['Team', 'HomeGoalsConceded', 'AwayGoalsConceded', 'TotalGoalsConceded']], on='Team')
total_stats['AllTimeGoalDiff'] = total_stats['TotalGoals'] - total_stats['TotalGoalsConceded']

total_stats = total_stats.sort_values(by=['AllTimeGoalDiff'], ascending=False)

total_stats.head()


,Team,HomeGoalsScored,AwayGoalsScored,TotalGoals,HomeGoalsConceded,AwayGoalsConceded,TotalGoalsConceded,AllTimeGoalDiff
0,Manchester Utd,1250,993,2243,438,671,1109,1134
1,Arsenal,1207,898,2105,514,677,1191,914
2,Liverpool,1202,894,2096,475,719,1194,902
3,Chelsea,1141,870,2011,504,668,1172,839
5,Manchester City,1017,735,1752,475,626,1101,651


Team Records

In [41]:
def get_head_to_head_record(team1, team2, df):
    # Filter matches involving the two teams
    relevant_matches = df[((df['Home'] == team1) & (df['Away'] == team2)) | ((df['Home'] == team2) & (df['Away'] == team1))]
    
    # Initialize counters
    team1_wins, team2_wins, draws = 0, 0, 0
    
    # Iterate through the filtered matches to count wins, losses, and draws
    for index, row in relevant_matches.iterrows():
        if row['FTR'] == 'H':  # Home win
            if row['Home'] == team1:
                team1_wins += 1
            else:
                team2_wins += 1
        elif row['FTR'] == 'A':  # Away win
            if row['Away'] == team1:
                team1_wins += 1
            else:
                team2_wins += 1
        else:  # Draw
            draws += 1
    
    # Compile the record into a dictionary
    record = {
        f'{team1}_wins': team1_wins,
        f'{team2}_wins': team2_wins,
        'draws': draws
    }
    
    return record

# Example usage of the function (you can replace 'Liverpool' and 'Manchester Utd' with any other teams from the dataset)
record_example = get_head_to_head_record('Sunderland', 'Bolton', df)
record_example

{'Sunderland_wins': 8, 'Bolton_wins': 4, 'draws': 4}

Scrapping Club Logos

In [11]:
# Extract unique team names
unique_teams = df['Home'].unique()

# Create a new DataFrame
teams_df = pd.DataFrame(unique_teams, columns=['Team'])
teams_df['Team_Logo'] = ''

# Ensure the logos folder exists
logos_folder = '/Users/ojonyeagwu/Desktop/epl_data/assets/team_logos'
os.makedirs(logos_folder, exist_ok=True)

# Function to scrape and download team logos
def download_team_logo(team_name):
    # Format the Wikipedia search URL by appending "F.C." to the team name
    search_url = f"https://en.wikipedia.org/wiki/{team_name} F.C."

    try:
        # Fetch the page content
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the infobox image, trying different methods
        infobox = soup.find(lambda tag: tag.name == "table" and "infobox" in tag.get("class", []))
        if infobox:
            infobox_image = infobox.find('img')
        else:
            # Fallback: try to find any image in the article body that might be the logo
            infobox_image = soup.find('div', class_='mw-parser-output').find('img')

        if not infobox_image:
            raise Exception("No suitable logo image found")

        # Construct the full image URL
        image_url = f"https:{infobox_image['src']}"

        # Download the image
        image_response = requests.get(image_url)

        # Save the image
        image_path = os.path.join(logos_folder, f"{team_name.replace(' ', '_')}_logo.png")
        with open(image_path, 'wb') as file:
            file.write(image_response.content)

        print(f'{team_name} logo found')
        return image_path
    
    except Exception as e:
        print(f"Error downloading logo for {team_name}: {e}")
        return ''

# Iterate over teams and download logos
for index, row in teams_df.iterrows():
    team_name = row['Team']
    logo_path = download_team_logo(team_name)
    teams_df.at[index, 'Team_Logo'] = logo_path

# Optionally, save the DataFrame to a new CSV
#teams_df.to_csv('teams_with_logos.csv', index=False)

Coventry City logo found
Leeds United logo found
Sheffield Utd logo found
Crystal Palace logo found
Arsenal logo found
Ipswich Town logo found
Everton logo found
Southampton logo found
Chelsea logo found
Nott'ham Forest logo found
Manchester City logo found
Blackburn logo found
Wimbledon logo found
Tottenham logo found
Liverpool logo found
Aston Villa logo found
Oldham Athletic logo found
Middlesbrough logo found
Norwich City logo found
QPR logo found
Manchester Utd logo found
Sheffield Weds logo found
Newcastle Utd logo found
West Ham logo found
Swindon Town logo found
Leicester City logo found
Bolton logo found
Sunderland logo found
Derby County logo found
Barnsley logo found
Charlton Ath logo found
Watford logo found
Bradford City logo found
Fulham logo found
Birmingham City logo found
West Brom logo found
Portsmouth logo found
Wolves logo found
Wigan Athletic logo found
Reading logo found
Hull City logo found
Stoke City logo found
Burnley logo found
Blackpool logo found
Swansea Cit

In [13]:
# Function to remove the specified path prefix from each 'Team_Logo' value
def remove_path_prefix(filepath):
    prefix = '/Users/ojonyeagwu/Desktop/epl_data'
    if filepath.startswith(prefix):
        # Remove the prefix and return the modified path
        return filepath[len(prefix):]
    return filepath  # Return the original path if the prefix is not found

# Apply the function to each value in the 'Team_Logo' column
teams_df['Team_Logo'] = teams_df['Team_Logo'].apply(remove_path_prefix)

In [14]:
teams_df

,Team,Team_Logo
0,Coventry City,/assets/team_logos/Coventry_City_logo.png
1,Leeds United,/assets/team_logos/Leeds_United_logo.png
2,Sheffield Utd,/assets/team_logos/Sheffield_Utd_logo.png
3,Crystal Palace,/assets/team_logos/Crystal_Palace_logo.png
4,Arsenal,/assets/team_logos/Arsenal_logo.png
5,Ipswich Town,/assets/team_logos/Ipswich_Town_logo.png
6,Everton,/assets/team_logos/Everton_logo.png
7,Southampton,/assets/team_logos/Southampton_logo.png
8,Chelsea,/assets/team_logos/Chelsea_logo.png
9,Nott'ham Forest,/assets/team_logos/Nott'ham_Forest_logo.png


In [16]:
teams_df.to_csv('/Users/ojonyeagwu/Desktop/epl_data/data/team_logos.csv', index=False)

In [10]:
# Define a function to calculate points
def calculate_points(row):
    if row['FTR'] == 'H':
        row['HomePoints'] = 3
        row['AwayPoints'] = 0
    elif row['FTR'] == 'A':
        row['HomePoints'] = 0
        row['AwayPoints'] = 3
    else:
        row['HomePoints'] = 1
        row['AwayPoints'] = 1
    return row

# Apply the function to the dataframe
data = df.apply(calculate_points, axis=1)

# Calculate goal difference
data['HomeGoalDifference'] = data['HomeGoals'] - data['AwayGoals']
data['AwayGoalDifference'] = data['AwayGoals'] - data['HomeGoals']

# Aggregate the points and goal difference for each team for each season
home_stats = data.groupby(['Season_End_Year', 'Home']).agg({'HomePoints': 'sum', 'HomeGoalDifference': 'sum'}).reset_index()
away_stats = data.groupby(['Season_End_Year', 'Away']).agg({'AwayPoints': 'sum', 'AwayGoalDifference': 'sum'}).reset_index()

# Merge the home and away stats
home_stats.rename(columns={'Home': 'Team', 'HomePoints': 'Points', 'HomeGoalDifference': 'GoalDifference'}, inplace=True)
away_stats.rename(columns={'Away': 'Team', 'AwayPoints': 'Points', 'AwayGoalDifference': 'GoalDifference'}, inplace=True)
total_stats = pd.concat([home_stats, away_stats])
final_stats = total_stats.groupby(['Season_End_Year', 'Team']).agg({'Points': 'sum', 'GoalDifference': 'sum'}).reset_index()

# Find the best and worst seasons
best_season = final_stats.loc[final_stats['Points'].idxmax()]
worst_season = final_stats.loc[final_stats['Points'].idxmin()]

best_season, worst_season


(Season_End_Year               2018
 Team               Manchester City
 Points                         100
 GoalDifference                  79
 Name: 516, dtype: object,
 Season_End_Year            2008
 Team               Derby County
 Points                       11
 GoalDifference              -69
 Name: 312, dtype: object)

In [14]:
import base64

df = pd.read_csv("/Users/ojonyeagwu/Desktop/epl_data/data/premier-league-matches.csv")
logos_df = pd.read_csv("/Users/ojonyeagwu/Desktop/epl_data/data/team_logos.csv")

def get_image_data_url(img_path):
    with open(img_path, "rb") as img_file:
        return f"data:image/png;base64,{base64.b64encode(img_file.read()).decode()}"

def titles_df():

    # Setup for the points system and calculation
    points = {
        'H': (3, 0),
        'D': (1, 1),
        'A': (0, 3)
    }

    df['HomePoints'] = df['FTR'].map(lambda x: points[x][0])
    df['AwayPoints'] = df['FTR'].map(lambda x: points[x][1])

    home_points = df.groupby(['Season_End_Year', 'Home'])['HomePoints'].sum().reset_index()
    away_points = df.groupby(['Season_End_Year', 'Away'])['AwayPoints'].sum().reset_index()

    home_points.rename(columns={'Home': 'Team', 'HomePoints': 'Points'}, inplace=True)
    away_points.rename(columns={'Away': 'Team', 'AwayPoints': 'Points'}, inplace=True)

    total_points = pd.concat([home_points, away_points])
    total_points = total_points.groupby(['Season_End_Year', 'Team'])['Points'].sum().reset_index()

    champions = total_points.loc[total_points.groupby('Season_End_Year')['Points'].idxmax()]
    titles_count = champions['Team'].value_counts().reset_index()
    titles_count.columns = ['Team', 'Titles']

    titles_with_logos = pd.merge(titles_count, logos_df, on='Team', how='left')

    titles_with_logos['Team_Logo'] = titles_with_logos['Team_Logo'].apply(get_image_data_url)

    # Reorder columns to make 'Team_Logo' the first column for titles_with_logos
    cols_titles = list(titles_with_logos.columns)
    cols_titles.insert(0, cols_titles.pop(cols_titles.index('Team_Logo')))  # Move 'Team_Logo' to first position
    titles_with_logos = titles_with_logos[cols_titles]

    return titles_with_logos

Scrapping New Data

In [6]:
import requests
import pandas as pd
import datetime

# Get the current year using datetime
current_year = datetime.datetime.now().year

original_data = pd.read_csv("/Users/ojonyeagwu/Desktop/epl_data/data/premier-league-matches.csv")

# Fetch team information from the bootstrap-static endpoint
def fetch_teams():
    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    response = requests.get(url)
    data = response.json()
    teams = {team['id']: team['name'] for team in data['teams']}
    return teams

# Fetch fixtures data
def fetch_fixtures():
    url = 'https://fantasy.premierleague.com/api/fixtures/'
    response = requests.get(url)
    fixtures = response.json()
    return fixtures

# Main data processing function
def process_fixtures():
    teams = fetch_teams()
    fixtures = fetch_fixtures()
    data = []

    for fixture in fixtures:
        if 'kickoff_time' in fixture and fixture['team_h_score'] is not None and fixture['team_a_score'] is not None:
            game = {
                'Season_End_Year': current_year if pd.to_datetime(fixture['kickoff_time']).year >= current_year else pd.to_datetime(fixture['kickoff_time']).year + 1,
                'Wk': fixture.get('event'),
                'Date': pd.to_datetime(fixture['kickoff_time']).date(),
                'Home': teams.get(fixture['team_h'], 'Unknown'),
                'HomeGoals': fixture['team_h_score'],
                'AwayGoals': fixture['team_a_score'],
                'Away': teams.get(fixture['team_a'], 'Unknown'),
                'FTR': 'H' if fixture['team_h_score'] > fixture['team_a_score'] else ('A' if fixture['team_h_score'] < fixture['team_a_score'] else 'D')
            }
            data.append(game)

    return pd.DataFrame(data)

# Execute the processing function and print the DataFrame
df = process_fixtures()
df


,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,2024,1,2023-08-11,Burnley,0,3,Man City,A
1,2024,1,2023-08-12,Arsenal,2,1,Nott'm Forest,H
2,2024,1,2023-08-12,Bournemouth,1,1,West Ham,D
3,2024,1,2023-08-12,Brighton,4,1,Luton,H
4,2024,1,2023-08-12,Everton,0,1,Fulham,A
...,...,...,...,...,...,...,...,...
318,2024,33,2024-04-13,Bournemouth,2,2,Man Utd,D
319,2024,33,2024-04-14,Liverpool,0,1,Crystal Palace,A
320,2024,33,2024-04-14,West Ham,0,2,Fulham,A
321,2024,33,2024-04-14,Arsenal,0,2,Aston Villa,A


In [7]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
data = response.json()
teams = {team['id']: team['name'] for team in data['teams']}
teams

{1: 'Arsenal',
 2: 'Aston Villa',
 3: 'Bournemouth',
 4: 'Brentford',
 5: 'Brighton',
 6: 'Burnley',
 7: 'Chelsea',
 8: 'Crystal Palace',
 9: 'Everton',
 10: 'Fulham',
 11: 'Liverpool',
 12: 'Luton',
 13: 'Man City',
 14: 'Man Utd',
 15: 'Newcastle',
 16: "Nott'm Forest",
 17: 'Sheffield Utd',
 18: 'Spurs',
 19: 'West Ham',
 20: 'Wolves'}

In [8]:
updated_df = pd.concat([original_data, df], ignore_index=True)

# Define a dictionary with the names you want to replace
replacement_dict = {
    'Man City': 'Manchester City',
    'Man Utd': 'Manchester Utd',
    'Newcastle': 'Newcastle Utd',
    "Nott'm Forest": "Nott'ham Forest",
    'Spurs': 'Tottenham'
}

# Replace the names in the "Home" and "Away" columns
updated_df['Home'] = updated_df['Home'].replace(replacement_dict, regex=True)
updated_df['Away'] = updated_df['Away'].replace(replacement_dict, regex=True)

updated_df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
...,...,...,...,...,...,...,...,...
12344,2024,33,2024-04-13,Bournemouth,2,2,Manchester Utd,D
12345,2024,33,2024-04-14,Liverpool,0,1,Crystal Palace,A
12346,2024,33,2024-04-14,West Ham,0,2,Fulham,A
12347,2024,33,2024-04-14,Arsenal,0,2,Aston Villa,A


In [9]:
gbq_test_df = updated_df[updated_df['Season_End_Year']<2024]
gbq_test_df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A


In [10]:
from pandas.io import gbq

In [12]:
gbq_test_df.to_gbq(destination_table='eplanalysis.epl_data',
                   project_id='eplanalysis',
                   if_exists='replace')

/var/folders/9v/27bzhgtx5kq5nwmqxjsf_fkw0000gn/T/ipykernel_40370/4158706599.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  gbq_test_df.to_gbq(destination_table='eplanalysis.epl_data',


In [31]:
arsenal_test_df = updated_df[updated_df.Season_End_Year == 2024]
arsenal_test_df = arsenal_test_df[updated_df.Home == 'Arsenal']
arsenal_test_df['Date'] = arsenal_test_df['Date'].astype(str)
arsenal_test_df 

/var/folders/9v/27bzhgtx5kq5nwmqxjsf_fkw0000gn/T/ipykernel_40370/2346548906.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  arsenal_test_df = arsenal_test_df[updated_df.Home == 'Arsenal']


,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
12027,2024,1,2023-08-12,Arsenal,2,1,Nott'ham Forest,H
12047,2024,3,2023-08-26,Arsenal,2,2,Fulham,D
12064,2024,4,2023-09-03,Arsenal,3,1,Manchester Utd,H
12080,2024,6,2023-09-24,Arsenal,2,2,Tottenham,D
12105,2024,8,2023-10-08,Arsenal,1,0,Manchester City,H
12118,2024,10,2023-10-28,Arsenal,5,0,Sheffield Utd,H
12137,2024,12,2023-11-11,Arsenal,3,1,Burnley,H
12156,2024,14,2023-12-02,Arsenal,2,1,Wolves,H
12191,2024,17,2023-12-17,Arsenal,2,0,Brighton,H
12213,2024,19,2023-12-28,Arsenal,0,2,West Ham,A


In [32]:
arsenal_test_df.to_gbq(destination_table='eplanalysis.epl_data',
                   project_id='eplanalysis',
                   if_exists='append')

/var/folders/9v/27bzhgtx5kq5nwmqxjsf_fkw0000gn/T/ipykernel_40370/4261859320.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  arsenal_test_df.to_gbq(destination_table='eplanalysis.epl_data',
